**Introduction**

TensorFlow includes a visualization tool called TensorBoard. It allows you to visualize TensorFlow graphs, histograms of variables at each step of the learning process, and more. You will annotate the neural network with operations to log summaries while the network is being trained. You will then use TensorBoard to read the logs and generate visualizations of the graph and learning process.

The summaries read by TensorBoard are created using the tf.summary module. There are functions in the module to write summaries for tensors (scalar), histograms, audio, and images. You will write summaries for tensors and histograms, but the other summary types are useful when working with audio or image data. Recall that TensorFlow uses a deferred execution model by first building a graph and then evaluating the graph to obtain results. Because summary operations aren't depended upon for computing results, they are only logging state information, the summary operations need to be explicitly evaluated in addition to evaluating output of your graph. The summary module includes a merge_all function to merge all summary operations into a single operation that you can conveniently evaluate instead of evaluating all individual summary operations. You will see how to evaluate both the summaries and the model training in the example code below.

In [1]:
'''Single neuron neural network with TensorBoard annotations'''

from __future__ import print_function
import numpy as np
import tensorflow as tf

# Start with a clean environment each run
log_dir = '/tmp/tensorflow/nn/train' # TensorBoard logs saved here 
if tf.gfile.Exists(log_dir):
  tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)
tf.reset_default_graph()

def variable_summaries(name, var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('weights'):
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

# Set up sample points perturbed away from the ideal linear relationship 
# y = 0.5*x + 2.5
num_examples = 60
points = np.array([np.linspace(-1, 5, num_examples),
  np.linspace(2, 5, num_examples)])
points += np.random.randn(2, num_examples)
x, y = points
# Include a 1 to use as the bias input for neurons
x_with_bias = np.array([(1., d) for d in x]).astype(np.float32)

# Training parameters
training_steps = 100
learning_rate = 0.001
losses = []

with tf.Session() as sess:
  # Set up all the tensors, variables, and operations.
  input = tf.constant(x_with_bias)
  target = tf.constant(np.transpose([y]).astype(np.float32))
  # Initialize weights with small random values
  weights = tf.Variable(tf.random_normal([2, 1], 0, 0.1))

  tf.global_variables_initializer().run()

  # Calculate the current prediction error
  y_predicted = tf.matmul(input, weights)
  y_error = tf.subtract(y_predicted, target)

  # Compute the L2 loss function of the error
  loss = tf.nn.l2_loss(y_error)
 
  # Train the network using an optimizer that minimizes the loss function
  update_weights = tf.train.GradientDescentOptimizer(
  learning_rate).minimize(loss)

  # Add summary operations
  variable_summaries('weights', weights)
  tf.summary.histogram('y_error', y_error)
  tf.summary.scalar('loss', loss)
  merged = tf.summary.merge_all()
  summary_writer = tf.summary.FileWriter(log_dir, graph=tf.get_default_graph())
 
  for i in range(training_steps):
    # Repeatedly run the summary and training operations
    summary, updates = sess.run([merged, update_weights])
    summary_writer.add_summary(summary, i)

  summary_writer.close()

print('Complete')

Instructions for updating:
Colocations handled automatically by placer.
Complete


**The code varies from the previous Lab Step's mainly by:**

- Including a function called variable_summaries that adds summary operations for the mean, min, max, standard deviation, and histogram of a tensor's values
- Including code to add summary operations and write the summaries to disk in the log_dir directory every iteration of the for loop (The code under the # Add summary operations comment)
- Removing the matplotlib plotting code since TensorBoard will create the visualizations automatically from the summary operations

You use a new way to evaluate the graph with this code. The following line is used to evaluate the graph:

    summary, updates = sess.run([merged, update_weights])

The session's run function evaluates the operations include in its argument, in this case, the merged summary and the update_weights training operations. All of the graph dependencies of the operations are also evaluated resulting in evaluating all of the graph operations.

Command to start tensorband: 
export LC_ALL="en_US.UTF-8"
export LC_CTYPE="en_US.UTF-8"
tensorboard --logdir /tmp/tensorflow/nn
    

![tensorflow3.png](attachment:image.png)